# Paper Analyses

Replication of the analyses in the paper: TBC

In [49]:
import os
import sys
import time

sys.path.append(os.path.join("..", "..", "DeepMet", "src"))
from workflow.training import train_single_model
from utils.feature_processing import get_fingerprints_from_meta, select_features
from datasets.main import load_dataset
from utils.config import Config

Compounds were extracted from the HMDB and ZINC12 databases, subject to the following constraints:
- Exact mass filter: 100Da < exact mass \> 800Da
- Other things

The entire set of compounds passing these filters in HMDB were retained while a random sample of 20,000 compounds were taken from ZINC12. The smiles for these compounds are available in the `data/test_set` folder.

In [25]:
data_path = "../data/test_set/"

# Location of the "normal" and "non-normal" smiles
normal_meta_path = os.path.join(data_path, "hmdb_meta.csv")
non_normal_meta_path = os.path.join(data_path, "zinc_meta.csv")

# Path to write results
results_path = "../paper_results"

if not os.path.exists(results_path):
    os.mkdir(results_path)

The function `train_likeness_scorer` implements the workflow for training the DeepMet model.
For the purposes of this vignette, the individual steps will be carried out manually.

While smiles are provided in the data files, these are not used as input to the model.
If not given to the `train_likeness_scorer` function, these will be converted to molecular
fingerprints using the smiles given as input. These are calculated in the following chunk.

This is a particularly time-consuming step, so it is recommended not to unnecessarily regenerate the fingerprints.

In [26]:
# normal_fingerprints_path = get_fingerprints_from_meta(normal_meta_path, os.path.join(results_path, "normal_fingerprints.csv"))
# non_normal_fingerprints_path = get_fingerprints_from_meta(non_normal_meta_path, os.path.join(results_path, "non_normal_fingerprints.csv"))

normal_fingerprints_path = os.path.join(results_path, "normal_fingerprints.csv")
non_normal_fingerprints_path = os.path.join(results_path, "non_normal_fingerprints.csv")

Here, we set the seed and set the training options for training DeepMet. The learning rate was
selected that was associated with the minimum loss on the validation set.

In [33]:
# Seed to be used for loading the dataset and training models
seed = 1

# Settings required by the DeepMet model
cfg = Config({
    "net_name": "cocrystal_transformer",
    "objective": "soft-boundary",
    "nu": 0.1,
    "rep_dim": 200,
    "seed": seed,
    "optimizer_name": "amsgrad",
    "lr": 0.000100095,
    "n_epochs": 20,
    "lr_milestones": tuple(),
    "batch_size": 2000,
    "weight_decay": 1e-5,
    "pretrain": False,
    "in_features": 2800,
    "device": "cpu"
})

While we have now generated the molecular fingerprints, these include many poorly balanced and
redundant features. We therefore use `select_features` to remove redundant and unbalanced features
prior to model training.

The data is then loaded into a torch-compatible format using `load_dataset`.

In [31]:
normal_fingerprints_path, non_normal_fingerprints_paths = select_features(
        normal_fingerprints_path=normal_fingerprints_path,
        normal_fingerprints_out_path=os.path.join(results_path, "selected_normal_fingerprints.csv"),
        non_normal_fingerprints_paths=non_normal_fingerprints_path,
        non_normal_fingerprints_out_paths=os.path.join(results_path, "selected_non_normal_fingerprints.csv")
)

# select_features allows for the simultaneous selection of multiple non-normal datasets
# we only have a single non-normal ZINC12 set here, which we will use to evaluate the final model
non_normal_fingerprints_path = non_normal_fingerprints_paths[0]

dataset, dataset_labels, validation_dataset = load_dataset(
    normal_dataset_path=normal_fingerprints_path,
    normal_meta_path=normal_meta_path,
    non_normal_dataset_path=non_normal_fingerprints_path,
    non_normal_dataset_meta_path=non_normal_meta_path,
    seed=seed,
    validation_split=0.8,
    test_split=0.9
)

KeyboardInterrupt: 

With the dataset loaded, we can now train the model. The core training workflow
is carried out using `train_single_model`. With the selected parameters, the final
AUC on the test set is 97.91% - importantly, AUC was not used for hyperparameter
optimisation as the validation set did not contain any "non-normal" compounds.

In [34]:
# Train the model (loss is calculated on the 'normal' validation set for parameter tuning)
deep_met_start = time.clock()
deep_met_model = train_single_model(cfg, validation_dataset)
deep_met_end = time.clock()

# Test using separate test dataset (includes the ZINC12 set of 'non-normal' compounds)
deep_met_model.test(dataset, device="cpu")

# AUC = 97.91%
print("AUC on test set: " + str(round(deep_met_model.results["test_auc"], 4)))

# Save model parameters
deep_met_model.save_model(os.path.join(results_path, "deep_met_model.tar"), False)

INFO:root:Set seed to 1.
INFO:root:Pretraining: False
INFO:root:Training optimizer: amsgrad
INFO:root:Training learning rate: 0.000100095
INFO:root:Training epochs: 20
INFO:root:Training batch size: 2000
INFO:root:Training weight decay: 1e-05
INFO:root:Initializing center c...
INFO:root:Center c initialized.
INFO:root:Starting training...
C:\Users\jackg\Miniconda3\envs\DeepMet\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
INFO:root:  Epoch 1/20	 Time: 9.282	 Loss: 73.16297208
INFO:root:  Epoch 2/20	 Time: 10.695	 L

Only one class present in y_true. ROC AUC score is not defined in that case.


AttributeError: 'NoneType' object has no attribute 'state_dict'

With DeepMet trained, we can train isolation forest and one-class SVM models for comparison. As for the DeepMet
model, non-normal compounds are not used for parameter selection. The contamination and nu parameters were set to 0.1
for consistency with DeepMet. The remaining isolation forest parameters and the OC-SVM kernel are the same
as were used for the Co-crystal paper. The gamma parameter was selected using the validation set and the scaled distance of the outliers
to the hyperplane as a loss function.

In [52]:
import pickle
from pyod.models import ocsvm, iforest

# iforest_model = iforest.IForest(
#     contamination=0.1,
#     n_estimators=400,
#     behaviour="new",
#     random_state=seed,
#     max_samples=1000
# )

ocsvm_model = ocsvm.OCSVM(
    contamination=0.1,
    kernel="rbf",
    nu=0.1,
    gamma=0.05
)

x_train = validation_dataset.train_set.dataset.data[validation_dataset.train_set.indices]

# iforest_start = time.clock()
# iforest_model.fit(x_train)
# iforest_end = time.clock()

ocsvm_start = time.time()
ocsvm_model.fit(x_train)
ocsvm_end = time.time()

# pickle.dump(iforest_model, open(os.path.join(results_path, "iforest_model.pkl"), "wb"))
pickle.dump(ocsvm_model, open(os.path.join(results_path, "ocsvm_model.pkl"), "wb"))

The isolation forests and OC-SVM models take a long time to train relative to DeepMet.

In [ ]:
print("DeepMet training time: " + str(deep_met_end - deep_met_start))
print("Isolation forests training time: " + str(iforest_end - iforest_start))
print("OC-SVM training time: " + str(ocsvm_end - ocsvm_start))

We can calculate AUC for these models as was done for DeepMet. Both the isolation forests and the OC-SVM
models have similar discriminative performance; they both have a lower AUC compared to DeepMet.

In [53]:
from sklearn.metrics import roc_auc_score

x_test = dataset.test_set.dataset.data[dataset.test_set.indices]
labels_test = dataset.test_set.dataset.labels[dataset.test_set.indices]

# print("Isolation forest AUC: " + str(roc_auc_score(labels_test, iforest_model.decision_function(x_test))))
print("OC-SVM AUC: " + str(roc_auc_score(labels_test, ocsvm_model.decision_function(x_test))))

OC-SVM AUC: 0.99220562461156
